In [250]:
import httpx
import asyncio  # executar as requisições de forma asíncrona
import pandas as pd

from bs4 import BeautifulSoup
from unidecode import unidecode # remover acentos
from tqdm.asyncio import tqdm_asyncio  # barra de progresso

links = []
responses = []
with open('Atlantico_Sul_N_NE.txt', encoding='utf-8') as file_:
    links = file_.read().split('\n')

async with httpx.AsyncClient() as client:
    tasks = [] # list de tarefas assíncronas
    for link in links[:20]:  # testando 20 requisições
        tasks.append(
            asyncio.create_task(client.get(link))
        )
    
    #aguardar todas as tarefas acabarem
    await tqdm_asyncio.gather(*tasks)
    responses = [task.result() for task in tasks]

# 
ocorrencias = []
for response_idx, response in enumerate(responses):
    dataframes_list = []
    soup = BeautifulSoup(response.text, features='lxml')
    # todas as tabelas
    tabelas = soup.select('#newsContent')

    ##########
    # tabela 1
    ##########

    columns = []
    data = []
    for table_row in tabelas[0].select('table')[0].select('tr'):
        try:
            table_data = table_row.select('td')
            if table_data[0].has_attr('bgcolor'):
                # remover acentos e por em caixa alta
                column_name = unidecode(table_data[0].get_text(strip=True).upper())
                # remover espacos e dois pontos:
                column_name = column_name.replace(" ","_").replace(":", "")
                # usamos get_text(strip=True) para evitar hex encoding (tipo \xa0)
                columns.append(column_name)
                data.append(table_data[1].get_text(strip=True))
        except IndexError:
            continue

    dataframes_list.append(pd.DataFrame([data], columns=columns))

    ##########
    # tabela 2
    ##########
    sub_tabelas = tabelas[1].select('td')[1].select("table")
    for sub_tabela in sub_tabelas:
        rows = sub_tabela.select('tr')
        if "Entrada d'água:" in rows[0].text:
            table_data = rows[1].select('td')
            columns = [table_data[0].get_text(strip=True)]
            data = [table_data[1].get_text(strip=True)]
            try:
                dataframes_list.append(pd.DataFrame([data], columns=columns))
            except (AssertionError, ValueError):
                continue
        else:
            if len(rows) > 1:
                table_data = rows[0].select('td')
                if table_data:
                    suffix = table_data[0].get_text(strip=True)
                    columns = []
                    data = []
                    for table_data_columns, table_data_rows in zip(rows[1], rows[2]):
                        name = table_data_columns.get_text(strip=True)
                        value = table_data_rows.get_text(strip=True)
                        if name != '':
                            column_name = unidecode(f"{suffix}_{name}".upper())
                            column_name = column_name.replace(" ","_").replace(":", "")
                            columns.append(column_name)
                            data.append(value)
                    dataframes_list.append(pd.DataFrame([data], columns=columns))

    ##########
    # tabela 3
    ##########
    # os dados estão apenas no segundo element td
    table_data = tabelas[2].select('tr')[0].select('td')[1]
    columns_elements = table_data.select('font')
    table_elements = table_data.select('table')
    for col_elem, table_elem in zip(columns_elements, table_elements):
        suffix = col_elem.get_text(strip=True)
        tr = table_elem.select('tr')
        if len(tr) == 1:
            td = tr[0].select('td')
            name = f"{suffix}_{td[0].get_text(strip=True)}"
            column = unidecode(name).replace(' ', '_').replace(':', '').upper()
            value = td[1].get_text(strip=True)
            dataframes_list.append(pd.DataFrame([value], columns=(column,)))
        else:
            names = tr[0].select('td')
            values = tr[1].select('td')
            columns = []
            data = []
            for name in names:
                name = f"{suffix}_{name.get_text(strip=True)}"
                name = name.replace(' ', '_').replace(':', '').upper()
                columns.append(unidecode(name))
            for value in values:
                value = f"{suffix}_{value.get_text(strip=True)}"
                data.append(value)
            dataframes_list.append(pd.DataFrame([data], columns=columns))


    ##########
    # tabela 4
    ##########
    # os dados estão apenas no segundo element td
    table_data = tabelas[3].select('tr')[0].select('td')[1]
    columns_elements = table_data.select('font')
    table_elements = table_data.select('table')

    suffix = columns_elements[0].get_text(strip=True)

    # tem um dado em células deslocadas (Aquífero)
    offset_row = table_elements[0].select('tr')

    # célula da esquerda
    offset_column = offset_row[0].select('td')[0]
    column, data = offset_column.get_text(strip=True).split(': ')
    column = unidecode(f"{suffix}_{column}").replace(' ', '').upper()
    dataframes_list.append(pd.DataFrame([data], columns=(column,)))

    # a segunda célula desloada (à direita de Aquífero)
    columns = []
    data = []
    for row in offset_row[0].select('td')[1].select('tr'):
        column_element, data_element = row.select('td')
        
        column = f"{suffix}_{column_element.get_text(strip=True)}"
        column = unidecode(column).upper()
        columns.append(column)
        data.append(data_element.get_text(strip=True))

    dataframes_list.append(pd.DataFrame([data], columns=columns))

    # tabela abaixo de Aquífero
    table_rows = table_elements[1].select('tr')
    suffix = columns_elements[1].get_text(strip=True)
    columns = []
    data = []
    for table_row in table_rows:
        column, value = table_row.select('td')
        column = f"{suffix}_{column.get_text(strip=True)}"
        column = column.replace(' ', '_').replace(':', '').upper()
        columns.append(column)
        data.append(value.get_text(strip=True))

    dataframes_list.append(pd.DataFrame([data], columns=columns))

    ##########
    # tabela 5
    ##########
    # os dados estão apenas no segundo element td
    table_data = tabelas[4].select('tr')[0].select('td')[1]
    suffix_element = table_data.select('font')
    table_elements = table_data.select('table')
    suffix = suffix_element[0].get_text(strip=True)

    table_rows = table_elements[0].select('tr')
    columns = []
    data = []
    # aqui acabei percebendo um jeito mais elegante de coletar
    # os valores nos elementos tr. Talvez dê pra o caso
    # das tabelas anteriores
    for table_row in table_rows[::2]:
        columns.extend([a.get_text(strip=True) for a in table_row.select('td')])
    for table_row in table_rows[1::2]:
        data.extend([a.get_text(strip=True) for a in table_row.select('td')])

    for idx, (col, value) in enumerate(zip(columns, data)):
        if col != '' and value != '':
            columns[idx] = f"{suffix}_{col}".replace(' ', '_').replace(':', '')
            columns[idx] = unidecode(columns[idx]).upper()

    dataframes_list.append(pd.DataFrame([data], columns=columns))

    ##########
    # tabela 6
    ##########
    # os dados estão apenas no segundo element td
    table_data = tabelas[5].select('tr')[0].select('td')[1]
    suffix_element = table_data.select('font')  # título da segunda subtabela
    table_elements = table_data.select('table')

    # 1a sub tabela
    suffix = ''
    for table_element in table_elements:
        columns = []
        data = []
        for idx, table_row in enumerate(table_element.select('tr')):
            if idx == 0:
                suffix = table_row.get_text(strip=True)
                continue
            table_data = table_row.select('td')
            unidade = ''
            if len(table_data) == 3:
                column, value, unidade = table_data
                unidade = unidade.get_text(strip=True).replace(' ', '_')
            else:
                column, value = table_data
            column = f"{suffix}_{column.get_text(strip=True)}".replace(' ', '_').replace(':', '')
            columns.append(unidecode(f"{column}_{unidade}").upper())
            data.append(value.get_text(strip=True))
            
        dataframes_list.append(pd.DataFrame([data], columns=columns))

    ocorrencias.append(pd.concat(dataframes_list, axis=1))

    # salva a cada 10 para liberar a memória
    if not (response_idx + 1)%10:
        fname = f"amostra_{response_idx - 9}_{response_idx+1}.parquet"
        dfs = [df.loc[:, ~df.columns.duplicated()].reset_index() for df in ocorrencias]
        amostra = pd.concat(dfs).iloc[:, 1:]
        amostra.to_parquet(fname, engine="pyarrow", index=False)
        ocorrencias = []
# como está o dataframe até agora?


100%|██████████| 20/20 [00:01<00:00, 13.05it/s]


In [251]:
from pathlib import Path

amostra = None
for parquet_file in Path().glob('*.parquet'):
    if amostra is None:
        amostra = pd.read_parquet(parquet_file)
        continue
    amostra = pd.concat([amostra, pd.read_parquet(parquet_file)])

# resetando os índices (eles ficam repetidos)
amostra = amostra.reset_index().iloc[:, 1:]
amostra
        

,NOME,DATA_DA_INSTALACAO,PROPRIETARIO,NATUREZA_DO_PONTO,USO_DA_AGUA,COTA_DO_TERRENO_(M),LOCALIDADE,UTM_(NORTE/SUL),UTM_(LESTE/OESTE),LATITUDE_(GGMMSS),...,PARAMETROCONCENTRACAOUNIDADE_NITRATOS_(NO3)_MG/L_(PPM),PARAMETROCONCENTRACAOUNIDADE_NITROGENIO_ORGANICO_MG/L_(PPM),PARAMETROCONCENTRACAOUNIDADE_ALCALINIDADE_TOTAL_MG/L_(PPM),PARAMETROCONCENTRACAOUNIDADE_FERRO_TOTAL_(FE)_MG/L_(PPM),PARAMETROCONCENTRACAOUNIDADE_NH3N_MG/L_(PPM),PARAMETROCONCENTRACAOUNIDADE_OXIGENIO_CONSUMIDO_EM_MEIO_BASICO_MG/L_(PPM),PARAMETROCONCENTRACAOUNIDADE_CARBONATO_(CO3)_MG/L_(PPM),PARAMETROCONCENTRACAOUNIDADE_NIQUEL_(NI)_MG/L_(PPM),TESTE_DE_BOMBEAMENTO_TIPO_DO_TESTE,TESTE_DE_BOMBEAMENTO_UNIDADE
0,XAN-001-PA,,Alberto César Jaime da Fonseca,Poço tubular,Abastecimento doméstico,0,Ananindeua,9847983,788245,012226,...,11,1.2,8,None,None,None,None,None,NaN,NaN
1,XAN-002-PA,05/04/1990,ANTONIO PEREIRA SOARES,Poço tubular,Abastecimento doméstico,0,Ananindeua,9847981,788345,012226,...,26,None,8,0.3,4.8,0.4,None,None,NaN,NaN
2,XAN-003-PA,20/05/1985,Manoel Lourenço da Costa,Poço tubular,Abastecimento doméstico,0,Ananindeua,9848017,788374,012225,...,18,None,None,-0.1,6.4,0.6,34,None,NaN,NaN
3,XAN-004-PA,22/01/1985,Enoque Vasconcelos Santos,Poço tubular,Abastecimento doméstico,0,Ananindeua,9847956,788374,012227,...,14,None,60,0.2,8,0.6,None,None,NaN,NaN
4,XAN-005-PA,22/03/1990,José Celestino da Luz Filho,Poço tubular,Abastecimento doméstico,0,Ananindeua,9847782,788250,012232,...,1,None,None,-0.1,5.2,0.8,4,None,NaN,NaN
5,XAN-006-PA,20/03/1989,Lucival Correa Lima,Poço tubular,Abastecimento doméstico,0,Ananindeua,9848077,788345,012223,...,14,5,6,0.2,None,None,None,None,NaN,NaN
6,XAN-007-PA,01/06/1993,Paulo Sérgio Araújo Vaz.,Poço tubular,Abastecimento doméstico,0,Ananindeua,9848079,788406,012223,...,14,4.8,6,0.2,None,0.4,None,None,NaN,NaN
7,XAN-008-PA,16/01/1994,Maria de Nazaré Moura Mendes.,Poço tubular,Abastecimento doméstico,0,Ananindeua,9848177,788349,012219,...,16,10,10,0.2,None,0.8,None,None,NaN,NaN
8,XAN-009-PA,02/11/1990,Estanes Charles,Poço tubular,Abastecimento doméstico,0,Ananindeua,9848181,789045,012219,...,9,None,None,-0.1,5.2,0.7,4,None,NaN,NaN
9,XAN-010-PA,01/03/1994,João Lucivaldo Rodrigues.,Poço tubular,Abastecimento doméstico,0,Ananindeua,9848177,788847,012219,...,8,None,None,0.2,2,0.6,None,0.005,NaN,NaN
